In [2]:
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [3]:
def suggest(category,product):
    client = MongoClient(port=27017)
    db=client.insurance
    cursor=db.quotes.aggregate([{"$group":{"_id":{"email":"$formData.email","category":"$category","product":"$product"}}},{"$project":{"email":"$_id.email","category":"$_id.category","product":"$_id.product","_id":0}}])
    data=pd.DataFrame(cursor)
    print(data)
    data["categoryProduct"]=data["category"]+","+data["product"]
    categoryProduct=set(data["categoryProduct"])
    outputData=list()
    for i in categoryProduct:
        iUsers=set(data.email[data["categoryProduct"]==i])    
        for j in categoryProduct:
            curOutput=list()
            curOutput.append(i)
            curOutput.append(j)
            jUsers=set(data.email[data["categoryProduct"]==j])
            curOutput.append(len(iUsers&jUsers))
            outputData.append(curOutput)
    outputData=pd.DataFrame(outputData,columns=["i","j","count"])
    categoryProduct=category+","+product
    processData=outputData[outputData["i"]==categoryProduct]
    #print(processData)
    print("length")
    print(len(set(data.email[data["categoryProduct"]==categoryProduct])))
    total=len(set(data.email[data["categoryProduct"]==categoryProduct]))
    if(total>0):
        processData["count"]=(processData["count"]*100)/total
        print(processData)
        categories=set(processData.j[processData["count"]>60])
        categoryList=list()
        productList=list()
        for cat in categories:
            split=cat.split(",")
            if(category!=split[0] and product!=split[1]):
                categoryList.append(split[0])
                productList.append(split[1])
        return list(db.formConfig.find({"category":{"$in":categoryList},"product":{"$in":productList}},{"category":1,"product":1,"image":1,"info":1,"_id":0}))
    return list()

In [4]:
suggest("Travel Insurance","Overseas Travel Insurance")

                            email             category  \
0       4sf15cs22@sahyadri.edu.in  Term Life Insurance   
1             ramanonda@gmail.com  Term Life Insurance   
2          prateek.jo97@gmail.com  Term Life Insurance   
3         mnrrnmproject@gmail.com      Motor Insurance   
4           attri.param@gmail.com     Travel Insurance   
5           attri.param@gmail.com  Term Life Insurance   
6                             NaN     Travel Insurance   
7      4sf15cs122@sahyadri.edu.in  Term Life Insurance   
8                             NaN     Health Insurance   
9       4sf15cs12@sahyadri.edu.in      Motor Insurance   
10          attri.param@gmail.com      Motor Insurance   
11            ramanonda@gmail.com      Motor Insurance   
12      4sf15cs12@sahyadri.edu.in     Travel Insurance   
13      4sf15cs12@sahyadri.edu.in  Term Life Insurance   
14            ramanonda@gmail.com     Travel Insurance   
15     4sf15cs122@sahyadri.edu.in      Motor Insurance   
16      4sf15c

<ipython-input-3-10718179d280>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processData["count"]=(processData["count"]*100)/total


[{'image': 'https://images.moneycontrol.com/static-mcnews/2018/03/car-insurance-770x433.jpg',
  'product': 'Car Insurance',
  'category': 'Motor Insurance',
  'info': ''},
 {'image': 'https://brandsite-static.hdfclife.com/media/image/45.jpg',
  'product': 'Individual Life Insurance',
  'category': 'Term Life Insurance',
  'info': ''}]

In [1]:
suggest("Term Life Insurance","Individual Life Insurance")

NameError: name 'suggest' is not defined

In [ ]:
suggest("Health Insurance","Individual Health Guard" )